In [ ]:
import pandas as pd
import numpy as np
import os
import sys
from sklearn.model_selection import train_test_split
import time


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras import models,layers,optimizers
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
DATASET_LOCATION_PATH='/home/nitin/workspace/local_datasets/xray-dataset'
TRAINING_IMG_LOCATION_PATH=f'{DATASET_LOCATION_PATH}/stage_2_train_images'
CHECKPOINT_LOCATION_PATH=f'{DATASET_LOCATION_PATH}/checkpoints'
MODEL_CHECKPOINT_LOCATION_PATH=f'{CHECKPOINT_LOCATION_PATH}/model_checkpoints'

In [ ]:
os.listdir(CHECKPOINT_LOCATION_PATH)

In [ ]:
X = np.load(f'{CHECKPOINT_LOCATION_PATH}/X.npy')
y = np.load(f'{CHECKPOINT_LOCATION_PATH}/y.npy')

In [ ]:
X.shape

In [ ]:
input_shape = (224,224,3)
nClasses = len(np.unique(y))
batch_size = 256
epochs = 50

In [ ]:
def createModel():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nClasses, activation='softmax'))
    
    return model

In [ ]:
stopping_callback = EarlyStopping(
    monitor='loss',
    patience=3
)
checkpoint_callback = ModelCheckpoint(
    filepath=f'{MODEL_CHECKPOINT_LOCATION_PATH}/model.{{epoch:02d}}-{{val_loss:.2f}}.h5',
    save_best_only=True,
    save_weights_only=True,
    verbose=1
)

In [ ]:
model = createModel()

In [ ]:
model.summary()

In [ ]:
plot_model(
    model,
    to_file='model.png',
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=False,
    dpi=96,
    layer_range=None,
    show_layer_activations=False,
    show_trainable=False
)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
start = time.time()
model.fit(x = X_train,
          y = y_train,
          validation_data = (X_test,y_test),
          batch_size = batch_size,
          epochs = epochs,
         )
end = time.time()
print(f"Runtime of the program is {end - start}")